# Deeplodocus

Проект по обучению моделей игре в шахматы


<p style="text-align:center;"><img src="logo.png" alt="drawing" width="200"/></p>

## Stockfish

In [10]:
import stockfish

stockfishEngine = stockfish.Stockfish(path="/home/sergey/stockfish/stockfish-ubuntu-x86-64-avx2")
stockfishEngine.set_elo_rating(1400)

## Deeplodocus engine

In [11]:
class Deeplodocus:
    @staticmethod
    def get_best_move(fen):
        return "e2e4"

## Dto

Класс для хранения dto-классов

In [12]:
class Dto:
    class ServerResponse:
        def __init__(self, move):
            self.move = move

    class ServerRequest:
        def __init__(self, fen, algorithm):
            self.fen = fen
            self.algorithm = algorithm

## AiRunner

Класс для запуска определённой модели по её имени

In [13]:
class AiRunner:
    @staticmethod
    def run(server_request):
        print("ai runner alg:", server_request.algorithm)
        print("ai runner: fen", server_request.fen)
        
        if str(server_request.algorithm) == "STOCKFISH":
            print("run STOCKFISH")
            print(stockfishEngine.is_fen_valid(server_request.fen))
            
            stockfishEngine.set_fen_position(server_request.fen)
            return Dto.ServerResponse(stockfishEngine.get_best_move())
        elif str(server_request.algorithm) == "DEEPLODOCUS":
            return Dto.ServerResponse(Deeplodocus.get_best_move(server_request.fen))

## Server

Класс для запуска локального сервера


In [14]:
import socket
import json


class Server:
    HOSTNAME = "localhost"
    PORT = 6671

    def __init__(self):
        self.server = socket.socket()
        self.server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

    def run(self):
        try:
            self.server.bind((self.HOSTNAME, self.PORT))
            self.server.listen()

            print("server starts")
            while True:
                con, addr = self.server.accept()
                json_str = con.recv(1024).decode('utf-8').replace('\n', '')

                print("server get data:", json_str)
                temp_obj = json.loads(json_str)

                server_req = Dto.ServerRequest(**temp_obj)
                server_res = AiRunner.run(server_req)

                message = json.dumps(server_res.__dict__)
                con.send(message.encode())

                print("server send message:", message)
                con.close()
        finally:
            self.server.close()
            print("server close")

## Запуск сервера

In [15]:
Server().run()

server starts
server close


KeyboardInterrupt: 